# 연습문제
1. 네이버 오픈 API 를 활용하여 네이버지식인 "전주여행" 과 "경주여행"을 검색
2. 명사를 추출
3. 빈도분석 (DataFrame)
4. 빈도시각화 (워드클라우드, nltk.Text)
5. Word2Vec




# 1. 네이버 open API 를 활용하여 검색 추출

- 검색어, no, title, link, description + title + '' + description

In [3]:
import os
import sys
import urllib.request
from dotenv import load_dotenv

load_dotenv()


client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

encText = urllib.parse.quote("전주여행")
url = f"https://openapi.naver.com/v1/search/kin.json?query={encText}&display=99" 


# JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Wed, 18 Jun 2025 19:37:02 +0900",
	"total":48152,
	"start":1,
	"display":99,
	"items":[
		{
			"title":"<b>전주<\/b> 가볼만한곳 추천 받아요",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=9&dirId=901&docId=486096029&qb=7KCE7KO87Jes7ZaJ&enc=utf8",
			"description":"... 추억의 7080 #다양한체험 #7080감성 #추억여행 #테마박물관 #유익한시간 2. [전북] 전북투어패스 통합이용권 #전북핫플 #여러여행지 #다양한체험 #카페이용추가 #<b>전주여행<\/b>필수 편안하고 즐거운 날이 되시길 바라겠습니다. 감사합니다 :)"
		},
		{
			"title":"<b>전주여행<\/b>갈려고하는데요!",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=9&dirId=90110&docId=472294467&qb=7KCE7KO87Jes7ZaJ&enc=utf8",
			"description":"<b>전주여행<\/b>을 갈려고하는데요!아는사람과 갈려고하는데 호텔은 좋은가격에 정했고~ 음..2박3일여행인데 얼마정도갖고가면좋을까요?그리고 맛집같은거 카페같은거 추천해주세요! 안녕하세요~ <b>전주 여행<\/b> 계획 중이시네요! 한옥마을 근처... "
		},
		{
			"title":"<b>전주<\/b> 1박 2일 <b>여행<\/b> 관련 질문~",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=9&dirId=90110&docId=483471047&qb=7KCE7KO87Jes7ZaJ&enc=utf8",
			"description":"... <b>전주 여행<\/b>을 계획하고 계시다니 정말 좋은 선택을 하셨네요. 전